# Fake News Predication

In [1]:
!pip install nltk

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fake : 1
# Real : 0

## downloding stopwords from englsih

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
##look at the stop words !!
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
print(type(stopwords))

<class 'nltk.corpus.reader.wordlist.WordListCorpusReader'>


## data pre-processing 

In [6]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [7]:
train_data.shape

(20800, 5)

In [8]:
test_data.shape

(5200, 4)

In [9]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [10]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Check for null values and fill them with empty string 

In [11]:
train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [12]:
train_data['title'] = train_data['title'].fillna('')
train_data['author'] = train_data['author'].fillna('')
train_data['text'] = train_data['text'].fillna('')

In [13]:
train_data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## merging author and title together to insted of processing text 

In [14]:
## space ' ' is because there should be space between words
train_data['combo'] = train_data['author'] + ' ' + train_data['title']

In [15]:
train_data.head()

,id,title,author,text,label,combo
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


## create combobination of author and title of test data

In [16]:
test_data['combo'] = test_data['author'] + ' ' + test_data['title']
test_data.head()

,id,title,author,text,combo
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David Streitfeld Specter of Trump Loosens Tong...
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,NaN
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams #NoDAPL: Native American Leaders...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",Daniel Victor Tim Tebow Will Attempt Another C...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast Network Keiser Report: Meme Wa...


## spliting features and labels data

In [24]:
train_features_set = train_data.drop('label',axis=1)
train_labels_set = train_data['label']

## making PorterStemmern obj 

In [26]:
port_stm = PorterStemmer()

## Steaming : process of converting anyword to its root word

In [39]:
def stemming(content):
    stem_content = re.sub('[^a-zA-Z]',' ',content)
    stem_content = stem_content.lower()
    stem_content = stem_content.split()
    stem_content = [port_stm.stem(word) for word in stem_content if word not in stopwords.words('english')]
    stem_content = ' '.join(stem_content)
    return stem_content

In [40]:
stemming('i am 15 years old dancing')

'year old danc'

## Passing Training features data through function

In [41]:
train_features_set['combo'] = train_features_set['combo'].apply(stemming)

In [43]:
train_features_set['combo'].head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: combo, dtype: object

## spaerating only combo coln and label coln for model training

In [53]:
x = train_features_set['combo'].values
y = train_labels_set.values

## using Vectorizer to convert text into vectors 

In [55]:
vectorizer = TfidfVectorizer()
vectorizer.fit(x)
x = vectorizer.transform(x)

In [59]:
print(x)

  (0, 267)	0.2701012497770876
  (0, 2483)	0.36765196867972083
  (0, 2959)	0.24684501285337127
  (0, 3600)	0.3598939188262558
  (0, 3792)	0.27053324808454915
  (0, 4973)	0.23331696690935097
  (0, 7005)	0.2187416908935914
  (0, 7692)	0.24785219520671598
  (0, 8630)	0.2921251408704368
  (0, 8909)	0.36359638063260746
  (0, 13473)	0.2565896679337956
  (0, 15686)	0.2848506356272864
  (1, 1497)	0.2939891562094648
  (1, 1894)	0.15521974226349364
  (1, 2223)	0.3827320386859759
  (1, 2813)	0.19094574062359204
  (1, 3568)	0.26373768806048464
  (1, 5503)	0.7143299355715573
  (1, 6816)	0.1904660198296849
  (1, 16799)	0.30071745655510157
  (2, 2943)	0.3179886800654691
  (2, 3103)	0.46097489583229645
  (2, 5389)	0.3866530551182615
  (2, 5968)	0.3474613386728292
  (2, 9620)	0.49351492943649944
  :	:
  (20797, 3643)	0.2115550061362374
  (20797, 7042)	0.21799048897828685
  (20797, 8364)	0.22322585870464115
  (20797, 8988)	0.36160868928090795
  (20797, 9518)	0.29542040034203126
  (20797, 9588)	0.17455348

## training the logestic regressor model

In [60]:
model = LogisticRegression()

In [61]:
model.fit(x,y)

LogisticRegression()

## Evaluation 

In [62]:
acc_scr = accuracy_score(model.predict(x),y)

In [63]:
acc_scr

0.988125

## Processing test data

In [68]:
test_data.isnull().sum()

id          0
title     122
author    503
text        7
combo     625
dtype: int64

In [69]:
test_data = test_data.fillna('')

In [70]:
test_data.isnull().sum()

id        0
title     0
author    0
text      0
combo     0
dtype: int64

In [71]:
test_data['author&title'] = test_data['author'] + ' ' + test_data['title']

In [72]:
test_data.head()

,id,title,author,text,combo,author&title
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",David Streitfeld Specter of Trump Loosens Tong...,David Streitfeld Specter of Trump Loosens Tong...
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists n...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,Common Dreams #NoDAPL: Native American Leaders...,Common Dreams #NoDAPL: Native American Leaders...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",Daniel Victor Tim Tebow Will Attempt Another C...,Daniel Victor Tim Tebow Will Attempt Another C...
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,Truth Broadcast Network Keiser Report: Meme Wa...,Truth Broadcast Network Keiser Report: Meme Wa...


In [73]:
test_feature = test_data['author&title']

In [75]:
test_feature = test_feature.apply(stemming)

In [77]:
test_feature

0       david streitfeld specter trump loosen tongu pu...
1       russian warship readi strike terrorist near al...
2       common dream nodapl nativ american leader vow ...
3       daniel victor tim tebow attempt anoth comeback...
4        truth broadcast network keiser report meme war e
                              ...                        
5195    jodi rosen bangladeshi traffic jam never end n...
5196    sheryl gay stolberg john kasich sign one abort...
5197    mike mcphate california today exactli sushi ne...
5198                us marin deploy russian border norway
5199        teddi wayn awkward sex onscreen new york time
Name: author&title, Length: 5200, dtype: object

In [78]:
test_vects = vectorizer.transform(test_feature)

In [80]:
print(test_vects)

  (0, 3623)	0.209228842739656
  (0, 8842)	0.36912420870480717
  (0, 10306)	0.08568694729690247
  (0, 12030)	0.38547947461491155
  (0, 13850)	0.32956045112215354
  (0, 14591)	0.34467447913058513
  (0, 14605)	0.3970837204511862
  (0, 15295)	0.08697876504498342
  (0, 15368)	0.38547947461491155
  (0, 15582)	0.11032513315099028
  (0, 16088)	0.3205594987716609
  (0, 16996)	0.08864595251126647
  (1, 347)	0.3487744590431239
  (1, 10219)	0.37818137822242154
  (1, 12300)	0.3838422559012773
  (1, 13048)	0.3052969733971102
  (1, 14603)	0.35559908473568963
  (1, 15142)	0.3550835494981517
  (1, 16473)	0.4921981982038152
  (2, 469)	0.20423466307498248
  (2, 2981)	0.2974392647613127
  (2, 4337)	0.2945337589015841
  (2, 5352)	0.29644778940921024
  (2, 8508)	0.30056564685986675
  (2, 8520)	0.24040124223443918
  :	:
  (5196, 15295)	0.08170410847188171
  (5196, 16198)	0.3273837588421952
  (5196, 16996)	0.08327019262492417
  (5197, 2184)	0.3217750895378809
  (5197, 5005)	0.4638294166150902
  (5197, 9405)	0

In [81]:
test_predications = model.predict(test_vects)

In [82]:
test_predications[:5]

array([0, 1, 1, 0, 1], dtype=int64)

In [83]:
## testing accuracy of test predications

In [84]:
test_predications

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [85]:
## pedcicting accuracy 

In [87]:
test_label = pd.read_csv('submit.csv')

In [91]:
test_label.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [93]:
test_acc = accuracy_score(test_predications,test_label['label'])

In [94]:
test_acc

0.6411538461538462